In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from   utilities3 import *
import operator
from   functools import reduce
from   functools import partial
from   timeit import default_timer
import scipy.io
import argparse

In [2]:
torch.manual_seed(30)
np.random.seed(30)

epochs          = 3000         
learning_rate   = 0.002 
scheduler_step  = 30
scheduler_gamma = 0.1
sub             = 1
ntrain          = 80
ntest           = 20
T_in            = 50
T_end           = 350
S               = 104
batch_size      = 1
mode1           = 12
mode2           = 12
width           = 60
step            = 1

In [3]:
################################################################
# fourier layer
################################################################
class SpectralConv2d_fast(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2):
        super(SpectralConv2d_fast, self).__init__()

        """
        2D Fourier layer. It does FFT, linear transform, and Inverse FFT.
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1 #Number of Fourier modes to multiply, at most floor(N/2) + 1
        self.modes2 = modes2

        self.scale = (1/(self.in_channels*self.out_channels*1000)) # here 10 means the dx and dz
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, \
                                                             self.modes1, self.modes2, dtype=torch.cfloat))
        self.weights2 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, \
                                                             self.modes1, self.modes2, dtype=torch.cfloat))

        self.weights3 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, \
                                                             self.modes1, self.modes2, dtype=torch.cfloat))
        self.weights4 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, \
                                                             self.modes1, self.modes2, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul2d(self, input, weights):
        # (batch, in_channel, x,y ), (in_channel, out_channel, x,y) -> (batch, out_channel, x,y)
        return torch.einsum("bixy,ioxy->boxy", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft_x = torch.fft.rfft2(x)
        x_ft_z = torch.fft.rfft2(torch.transpose(x,2,3))

        # Multiply relevant Fourier modes
        out_ft_x = torch.zeros(batchsize, self.out_channels,  x.size(-2), x.size(-1)//2 + 1, \
                             dtype=torch.cfloat, device=x.device)
        out_ft_z = torch.zeros(batchsize, self.out_channels,  x.size(-2), x.size(-1)//2 + 1, \
                             dtype=torch.cfloat, device=x.device)
        out_ft_x[:, :, :self.modes1, :self.modes2]  = self.compl_mul2d(x_ft_x[:, :, :self.modes1, :self.modes2], self.weights1)
        out_ft_x[:, :, -self.modes1:, :self.modes2] = self.compl_mul2d(x_ft_x[:, :, -self.modes1:, :self.modes2], self.weights2)


        out_ft_z[:, :, :self.modes1, :self.modes2]  = self.compl_mul2d(x_ft_z[:, :, :self.modes1, :self.modes2], self.weights3)
        out_ft_z[:, :, -self.modes1:, :self.modes2] = self.compl_mul2d(x_ft_z[:, :, -self.modes1:, :self.modes2], self.weights4)

        #Return to physical space
        out_ft_x = torch.fft.irfft2(out_ft_x, s=(x.size(-2), x.size(-1)))
        out_ft_z = torch.fft.irfft2(out_ft_z, s=(x.size(-2), x.size(-1)))
        #print("this is the shape of the out_x", out_ft_x.shape)
        #print("this is the shape of the out_z", out_ft_z.shape)
        
        return out_ft_x, out_ft_z

In [4]:

class FNO2d(nn.Module):
    def __init__(self, modes1, modes2, width):
        super(FNO2d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .

        input: the solution of the previous 10 timesteps + 2 locations (u(t-10, x, y), ..., u(t-1, x, y),  x, y)
        input shape: (batchsize, x=64, y=64, c=12)
        output: the solution of the next timestep
        output shape: (batchsize, x=64, y=64, c=1)
        """

        self.modes1 = modes1
        self.modes2 = modes2
        self.width = width
        self.fc0 = nn.Linear((50+5), self.width)
        # input channel is 12: the solution of the previous 10 timesteps + 2 locations (u(t-10, x, y), ..., u(t-1, x, y),  x, y)

        self.conv0 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.conv1 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.conv2 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.conv3 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.conv4 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.conv5 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)
        self.conv6 = SpectralConv2d_fast(self.width, self.width, self.modes1, self.modes2)


        self.w0 = nn.Conv2d(self.width, self.width, 1)
        self.w1 = nn.Conv2d(self.width, self.width, 1)
        self.w2 = nn.Conv2d(self.width, self.width, 1)
        self.w3 = nn.Conv2d(self.width, self.width, 1)
        self.w4 = nn.Conv2d(self.width, self.width, 1)
        self.w5 = nn.Conv2d(self.width, self.width, 1)
        self.w6 = nn.Conv2d(self.width, self.width, 1)

        self.bn0 = torch.nn.BatchNorm2d(self.width)
        self.bn1 = torch.nn.BatchNorm2d(self.width)
        self.bn2 = torch.nn.BatchNorm2d(self.width)
        self.bn3 = torch.nn.BatchNorm2d(self.width)
        self.bn4 = torch.nn.BatchNorm2d(self.width)
        self.bn5 = torch.nn.BatchNorm2d(self.width)
        self.bn6 = torch.nn.BatchNorm2d(self.width)

        self.fc1 = nn.Linear(self.width, 300)
        #self.fc2 = nn.Linear(200, args.step)

    def forward(self, x , C1, C2, C3):
        batchsize = x.shape[0]

        size_x, size_y = x.shape[1], x.shape[2]

        C1 = torch.reshape(C1,[batchsize,1,size_x,size_y])
        C2 = torch.reshape(C2,[batchsize,1,size_x,size_y])
        C3 = torch.reshape(C3,[batchsize,1,size_x,size_y])

        C1 = torch.nn.functional.normalize(C1)
        C2 = torch.nn.functional.normalize(C2)
        C3 = torch.nn.functional.normalize(C3)

        C1 = torch.reshape(C1,[batchsize,size_x,size_y,1])
        C2 = torch.reshape(C2,[batchsize,size_x,size_y,1])
        C3 = torch.reshape(C3,[batchsize,size_x,size_y,1])

        grid = self.get_grid(batchsize, size_x, size_y, x.device)
        '''
        print("this is the shape of x",     x.shape)
        print("this is the shape of grid ", grid.shape)
        print("this is the shape of C1 ",   C1.shape)
        print("this is the shape of C2 ",   C2.shape)
        print("this is the shape of C3 ",   C3.shape)
        '''
        x = torch.cat((x, grid, C1, C2, C3), dim=-1)
        #print("this is the shape od x ",x.shape)
        x = self.fc0(x)
        x = x.permute(0, 3, 1, 2)

        x0, z0 = self.conv0(x)
        xw_0   = self.w0(x)
        #print("this is the shape of the wx0", xw_0.shape)
        x      = self.bn0(xw_0 + x0 + z0)
        x = F.relu(x)
        
        x1, z1 = self.conv1(x)
        xw_1   = self.w1(x)
        x      = self.bn1(xw_1 + x1 + z1)
        x = F.relu(x)
        
        x2, z2 = self.conv2(x)
        xw_2   = self.w2(x)
        x      = self.bn2(xw_2 + x2 + z2)
        x = F.relu(x)
        
        
        x3, z3 = self.conv3(x)
        xw_3   = self.w3(x)
        x      = self.bn3(xw_3 + x3 + z3)
        x = F.relu(x)

        x = x.permute(0, 2, 3, 1)
        x = self.fc1(x)
        #x = F.relu(x)
        #x = self.fc2(x)

        return x

    def get_grid(self, batchsize, size_x, size_y, device):
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
        return torch.cat((gridx, gridy), dim=-1).to(device)

In [5]:
def tensor_pad(input_tensor,npad, freeSurface,dtype):
    """
    This function is to padding velocity tensor for implementing the absorbing boudary condition.
        input_tensor: is a 3D tensor, shape=[1, self.in_channels, self.nz, self.nx]
        output_tensor: is also a 3D tensor, shape=[1, self.in_channels, self.nz_pad, self.nx_pad]
    """
    nz,nx = input_tensor.shape[0],input_tensor.shape[1]
    input_tensor = input_tensor.reshape(1,1,nz,nx)
    #print("this is input tensor shape", input_tensor.shape)

    nz_pad = nz + 2* npad
    nx_pad = nx + 2* npad
    in_channels = 1

    if freeSurface:
        vpadTop = input_tensor
    else:
        vtop = torch.ones((1, in_channels, npad, nx),dtype = dtype) * input_tensor[:,:,0,:]
        vpadTop = torch.cat((vtop,input_tensor), -2) #padding on axis=2 (nz)

    vbottom = torch.ones((1, in_channels, npad, nx),dtype = dtype) * input_tensor[:,:,-1,:]
    vpadBottom = torch.cat([vpadTop,vbottom], -2)#padding on axis=2 (nz)

    vleft = torch.ones((1, in_channels, nz_pad, npad),dtype = dtype) * \
    vpadBottom[:,:,:,0].view(1, in_channels, -1, 1)
    vpadLeft = torch.cat([vleft,vpadBottom], -1)#padding on axis=3 (nx)

    vright = torch.ones((1, in_channels, nz_pad, npad),dtype= dtype) * \
    vpadBottom[:,:,:,-1].view(1,in_channels, -1, 1)
    output_tensor = torch.cat([vpadLeft,vright], -1)#padding on axis=3 (nx)

    return output_tensor

In [6]:
TRAIN_PATH = 'E:\FNO_fields\FN_small_model_various_source'
#TRAIN_PATH = '/Users/zhangtianze/Documents/FWI/fourier_neural_operator-master/ns_data.mat'
vx_train = torch.load(TRAIN_PATH+'/vx_save_0916_tnesor.pt')
#vz_train = torch.load("/home/zhang.tianze/fourier_neural_operator-master/vz_save.pt")

vp_index  = torch.load(TRAIN_PATH+'/vp_model_index_0916.pt')
vs_index  = torch.load(TRAIN_PATH+'/vs_model_index_0916.pt')
rho_index = torch.load(TRAIN_PATH+'/rho_model_index_0916.pt')

print("this is the shape of the original size",vp_index.shape)

vp_index = vp_index[0:100,:,:].reshape(100,64,64,1)
vs_index = vs_index[0:100,:,:].reshape(100,64,64,1)
rho_index = rho_index[0:100,:,:].reshape(100,64,64,1)

print(epochs, learning_rate, scheduler_step, scheduler_gamma)
print("this is the shape of vp vs and rho",vp_index.shape,vs_index.shape,rho_index.shape)

this is the shape of the original size torch.Size([100, 64, 64])
3000 0.002 30 0.1
this is the shape of vp vs and rho torch.Size([100, 64, 64, 1]) torch.Size([100, 64, 64, 1]) torch.Size([100, 64, 64, 1])


In [7]:
vp_index_reshape  = torch.zeros(100,104,104)
vs_index_reshape  = torch.zeros(100,104,104)
rho_index_reshape = torch.zeros(100,104,104)

for i_model_index in range(100):
    vp_index_reshape_i = tensor_pad(vp_index[i_model_index,:,:,:], 20, False, torch.float32)
    vp_index_reshape[i_model_index,:,:] = vp_index_reshape_i.squeeze()

    vs_index_reshape_i = tensor_pad(vs_index[i_model_index,:,:,:], 20, False, torch.float32)
    vs_index_reshape[i_model_index,:,:] = vs_index_reshape_i.squeeze()

    rho_index_reshape_i = tensor_pad(rho_index[i_model_index,:,:,:], 20, False, torch.float32)
    rho_index_reshape[i_model_index,:,:] = rho_index_reshape_i.squeeze()

In [8]:
vp_index_reshape =  vp_index_reshape.reshape(100,104,104,1)
vs_index_reshape =  vs_index_reshape.reshape(100,104,104,1)
rho_index_reshape = rho_index_reshape.reshape(100,104,104,1)

vp_index_reshape =  vp_index_reshape.reshape(100,104,104,1)
vs_index_reshape =  vs_index_reshape.reshape(100,104,104,1)
rho_index_reshape = rho_index_reshape.reshape(100,104,104,1)

In [9]:
runtime = np.zeros(2, )
t1 = default_timer()

In [10]:
print("this is the shape of vx_train", vx_train.shape)
#print("this is the shape of vx_train", vz_train.shape)
vx_train_reshape = vx_train.permute(0,2,3,1)

print(vx_train_reshape.shape)
#print(vz_train_reshape.shape)

this is the shape of vx_train torch.Size([100, 800, 104, 104])
torch.Size([100, 104, 104, 800])


In [11]:
train_a = vx_train_reshape[:ntrain,::sub,::sub,:T_in]
train_u = vx_train_reshape[:ntrain,::sub,::sub,T_in:T_end]

train_vp  =  vp_index_reshape[:ntrain,::sub,::sub,:]
train_vs  =  vs_index_reshape[:ntrain,::sub,::sub,:]
train_rho = rho_index_reshape[:ntrain,::sub,::sub,:]

test_a  = vx_train_reshape[ntrain:ntrain+ntest,::sub,::sub,:T_in]
test_u  = vx_train_reshape[ntrain:ntrain+ntest,::sub,::sub,T_in:T_end]

test_vp  = vp_index_reshape [ntrain:ntrain+ntest,::sub,::sub,:]
test_vs  = vs_index_reshape [ntrain:ntrain+ntest,::sub,::sub,:]
test_rho = rho_index_reshape[ntrain:ntrain+ntest,::sub,::sub,:]

In [12]:
print("this is the training_set of of a", train_a.shape)
print("this is the training_set of of u", train_u.shape)
print("this is the training_set of of vp",  train_vp.shape)
print("this is the training_set of of vs",  train_vs.shape)
print("this is the training_set of of rho", train_rho.shape)

print("this is the testing_set of of a", test_a.shape)
print("this is the testing_set of of u", test_u.shape)

print("this is the testing_set of of vp",  test_vp.shape)
print("this is the testing_set of of vs",  test_vs.shape)
print("this is the testing_set of of rho", test_rho.shape)

this is the training_set of of a torch.Size([80, 104, 104, 50])
this is the training_set of of u torch.Size([80, 104, 104, 300])
this is the training_set of of vp torch.Size([80, 104, 104, 1])
this is the training_set of of vs torch.Size([80, 104, 104, 1])
this is the training_set of of rho torch.Size([80, 104, 104, 1])
this is the testing_set of of a torch.Size([20, 104, 104, 50])
this is the testing_set of of u torch.Size([20, 104, 104, 300])
this is the testing_set of of vp torch.Size([20, 104, 104, 1])
this is the testing_set of of vs torch.Size([20, 104, 104, 1])
this is the testing_set of of rho torch.Size([20, 104, 104, 1])


In [13]:
print(train_u.shape)
assert (S            == train_u.shape[-2])
assert ((T_end-T_in) == train_u.shape[-1])

assert (S            == test_u.shape[-2])
assert ((T_end-T_in) == test_u.shape[-1])

assert (S == train_vp.shape[-2])
assert (S == train_vs.shape[-2])
assert (S == train_rho.shape[-2])

assert (S == test_vp.shape[-2])
assert (S == test_vs.shape[-2])
assert (S == test_rho.shape[-2])

torch.Size([80, 104, 104, 300])


In [14]:
train_a = train_a.reshape(ntrain, S, S,T_in)
train_u = train_u.reshape(ntrain, S, S,(T_end - T_in))
test_a  = test_a.reshape( ntest,  S, S,T_in)
test_u  = test_u.reshape( ntest,  S, S,(T_end - T_in))

In [15]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_a, train_u,\
                                                                          train_vp,train_vs,train_rho), \
                                                                          batch_size = batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u, \
                                                                          test_vp,test_vs,test_rho), \
                                                                          batch_size = batch_size, shuffle=True)

In [16]:
device = torch.device('cuda')
print("this is the devide we are using device \n", device)
model = FNO2d(mode1, mode2, width).cuda()

this is the devide we are using device 
 cuda


In [17]:
print(count_params(model))
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

training_loss_tensor = torch.zeros(epochs)
testing_loss_tensor  = torch.zeros(epochs)

myloss = LpLoss(size_average=False)
costFunc = torch.nn.MSELoss(reduction='sum')
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2_step_training = 0
    test_l2_step_testing = 0
    for xx, yy, C1, C2, C3 in train_loader:
        #print("this the the loading of training loader")
        loss = 0
        xx = xx.to(device)
        yy = yy.to(device)
        C1 = C1.to(device)
        C2 = C2.to(device)
        C3 = C3.to(device)

        for t in range(0, (T_end - T_in),step):
            y = yy[..., t:t + step]
            #print("this is the shape of y",y.shape)
            im_train = model(xx,C1,C2,C3)
            #loss += costFunc(im_train,y)
            loss += myloss(im_train.reshape(batch_size, -1), y.reshape(batch_size, -1))
            if t == 0:
                pred = im_train
            else:
                pred = torch.cat((pred, im_train), -1)

            xx = torch.cat((xx[..., step:], im_train), dim=-1)

            #print("this is the shape of xx ======>", xx.shape)
        #print("this before loss backpropagation")
        train_l2_step_training += loss.item()
        #l2_full = myloss(pred.reshape(batch_size, -1), yy.reshape(batch_size, -1))
        #train_l2_full += l2_full.item()

        optimizer.zero_grad()
        loss.backward()
        #print("this is the loss ====== >", loss)
        #print("this is the epoch====== >", ep)
        optimizer.step()
        training_loss_tensor[ep] = train_l2_step_training

    test_l2_step = 0
    test_l2_full = 0
    
    with torch.no_grad():
        for xx, yy, C1, C2, C3 in test_loader:
            loss = 0
            xx = xx.to(device)
            yy = yy.to(device)
            C1 = C1.to(device)
            C2 = C2.to(device)
            C3 = C3.to(device)

            for t in range(0, (T_end - T_in),step):
                y = yy[..., t:t + step]
                im_test = model(xx,C1,C2,C3)
                #loss += costFunc(im_test,y)
                loss += myloss(im_test.reshape(batch_size, -1), y.reshape(batch_size, -1))
                if t == 0:
                    pred = im_test
                else:
                    pred = torch.cat((pred, im_test), -1)

                xx = torch.cat((xx[..., step:], im_test), dim=-1)

            test_l2_step_testing += loss.item()
            #test_l2_full += myloss(pred.reshape(batch_size, -1), yy.reshape(batch_size, -1)).item()
            testing_loss_tensor[ep] = test_l2_step_testing

    t2 = default_timer()
    #scheduler.step()
    print("current epock",ep, \
          "processing time in each iteration [{:.4e}]".format(t2 - t1), \
          "training loss [{:.4e}]".format(train_l2_step_training), \
          "testing loss  [{:.4e}]".format(test_l2_step_testing))

14563320


RuntimeError: The size of tensor a (3244800) must match the size of tensor b (10816) at non-singleton dimension 1

In [ ]:
im_plot_train  = im_train[2,:,:,200].cpu().clone().detach().numpy()
im_plot_tests  = im_test [2,:,:,200].cpu().clone().detach().numpy()

In [ ]:
fig = plt.figure(figsize = (10,10)) # create the canvas for plotting
ax1 = plt.subplot(1, 2, 1)
ax1.imshow(im_plot_train, cmap='bwr_r')
ax2 = plt.subplot(1, 2, 2)
ax2.imshow(im_plot_tests, cmap='bwr_r')

In [ ]:
plt.plot(training_loss_tensor)
plt.yscale('log')

In [ ]:
plt.plot(testing_loss_tensor)
plt.yscale('log')